In [1]:
import numpy as np
from typing import TypeAlias
import time

from hyperparameters import (
    _dt,
    _t_max,
    _tau_m,
    _V_reset,
    _V_thresh,
    _R,
    num_simulations
)

with open('weights.npy', 'rb') as f:
    W = np.load(f)

import jax.numpy as jnp

# Initial conditions
n_neurons = len(W)# Number of neurons in the network
_V = jnp.ones(n_neurons) * _V_reset  # Initial potentials

from tqdm.notebook import tqdm

2024-04-18 10:30:01.341513: W external/xla/xla/service/gpu/nvptx_compiler.cc:718] The NVIDIA driver's CUDA version is 12.2 which is older than the ptxas CUDA version (12.4.131). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


# Type Definitions for Clarity

In [2]:
Tensor3D: TypeAlias = jnp.ndarray
Mat: TypeAlias = jnp.ndarray
Vec: TypeAlias = jnp.ndarray 

# Run the Simulations

In [3]:
def run_simulation(
    W: Mat,
    V: Vec,

    # Neuron Parameters
    tau_m: float,
    v_reset: float,
    v_thresh: float,
    membr_R: float,

    # How long do we run for? 
    t_max: float,
    dt: float, 

):
    # Simulation

    spike_train = []
    for i, t in enumerate(np.arange(0, t_max, dt)):
        if i == 0:
            continue

        fired = V >= v_thresh
        V = jnp.where(fired, v_reset, V)
        
        # Record spike times
        spike_train.append(fired)
    
        # Update voltages
        I_syn = W.dot(spike_train[-1])  # Synaptic current from spikes
        dV = (dt / tau_m) * (-V + v_reset + membr_R * I_syn)
        V = V + dV
    
        # No self-inputs; neurons cannot spike themselves in this timestep
        V = jnp.where(fired, v_reset, V)
    return spike_train

time_arr = []
for i in range(num_simulations):
    start = time.time()
    spike_train = run_simulation(
        W,
        _V,
        _tau_m, _V_reset, _V_thresh, _R,
        _t_max, _dt
    )
    end = time.time()
    print(f"Iteration {i} took: {end - start} seconds")
    time_arr.append(end - start)

print(f"Average Time: {np.mean(time_arr)}")
print(f"S.Dev Time: {np.std(time_arr)}")

Iteration 0 took: 0.8546652793884277 seconds
Iteration 1 took: 0.8406507968902588 seconds
Iteration 2 took: 0.7851078510284424 seconds
Iteration 3 took: 0.6649010181427002 seconds
Iteration 4 took: 0.6667559146881104 seconds
Iteration 5 took: 0.7485339641571045 seconds
Iteration 6 took: 0.6844122409820557 seconds


KeyboardInterrupt: 